# Apache Server on EC2


This lab will walk you through steps to launch Apache web server on an EC2 instance. 
You should have an AWS account created by now either from DSA or by yourself. 


Steps Overview:
 1. By the end of this lab you will be able to launch an EC2 instance using boto3 python
package. (You can refer to [Launch Web Server Practice](../../module1/practices/PracticeWebServer.ipynb) lab for a reference).
 1. You will be able to install any additional software you need on the instance.
 1. Install PHP and mysql softwares using terminal commands.
 1. We will upload a static webpage to the instance and display a simple message in the browser. 

In [ ]:
################################### SET THE FOLLOWING PARAMETERS ###################################################
#Set the AWS Region
region = 'us-east-1'


ami_image = 'ami-8c1be5f6'

#Set the AWS Access ID (Given to you buy the DSA staff: "Access key ID")
access_id = 'Put your access key id here'

#Set the AWS Access Key (Given to you buy the DSA staff: "Secret access key")
access_key = 'Put your secret access key here'

#Security group name: Add your pawprint here
Sec_group_name= "WebServer_your_pawprint_Sec_group"

In [ ]:
#Import AWS' Python Based DEVOPS tools
import boto3
from botocore.exceptions import ClientError

#Import System Tools
import collections
import json
import os
import datetime
import pandas
import time
import getpass
from subprocess import call

#Set the username from system
system_user_name=getpass.getuser()

# client interface.
# Estabilish Credentials/Session
ec2 = boto3.client(
    'ec2', 
    region_name=region,
    aws_access_key_id=access_id,
    aws_secret_access_key=access_key
)

### Create a Security Group

Create a security group and modify the security rules as we need to SSH into the instance to install software packages on it. . 

In [ ]:
sg = ec2.create_security_group(
    Description='security grp for web server',
    GroupName=Sec_group_name
)
Sec_group=sg["GroupId"]

Customize the security group to allow MU's TCP traffic and SSH requests. Configure the inbound rules to allow traffic as needed. 

In [ ]:
#Modify Security Configuration to allow MU's IP addresses

#Describe Cluster

try:
    sec_rule="ALL TCP"
    data = ec2.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': 0,
             'ToPort': 65535,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        ],)
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")
#     print(data)

try:
    sec_rule="ALL TCP"
    data = ec2.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': 0,
             'ToPort': 65535,
             'UserIdGroupPairs': [{ 'GroupId': Sec_group }]
#              'IpRanges': [{'CidrIp': Sec_group}]},
            }],
#         SourceSecurityGroup=Sec_group_name
    )
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")

try:
    sec_rule="Custom ICMP Rule - IPv4"
    data = ec2.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'icmp',
             'FromPort': 0,
             'ToPort': -1,
             'IpRanges': [{'CidrIp': '173.31.192.195/32'}]},
        ])
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")
#     print(data)

try:
    sec_rule="ALL UDP"
    data = ec2.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'udp',
             'FromPort': 0,
             'ToPort': 65535,
             'UserIdGroupPairs': [{ 'GroupId': Sec_group }]
            }],
    )
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")
#     print(data)

    
try:
    sec_rule="ALL ICMP"
    data = ec2.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'icmp',
             'FromPort': -1,
             'ToPort': -1,
             'UserIdGroupPairs': [{ 'GroupId': Sec_group }]
            }],
    )
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")

    
try:
    sec_rule="ALL ICMP"
    data = ec2.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'icmp',
             'FromPort': -1,
             'ToPort': -1,
             'IpRanges': [{'CidrIp': '0.0.0.0/16'}]
            }],
    )
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")


### Create a keypair

Create a keypair for the EC2 instance. We first generate a name to create a key with that name and also store the key in a file. ec2.create_key_pair() will create a keypair. System command echo is used to write the contents of keypair generated to a file created with same name as keypair. 

You have to modify the file permissions to provide readonly access. If the file is open, system will throw an error. Do chmod(file, 0o400) 

In [ ]:
import time 
import os

ec2_pem_file=time.strftime("EC2-%d%m%Y%H%M%S-"+system_user_name)
ec2_key=ec2.create_key_pair(KeyName=ec2_pem_file)

#Don't do this unless you have a good reason
#print(emr_key['KeyMaterial'])

os.system("echo \""+ec2_key['KeyMaterial']+"\" > "+ec2_pem_file+".pem")
os.chmod(ec2_pem_file+".pem",0o400)

print("KeyName         : "+ec2_key['KeyName']+"\nKey Fingerprint : "+ec2_key['KeyFingerprint'])

### Create Instance

In [ ]:
instances = ec2.run_instances(
    ImageId=ami_image,
    MinCount=1,
    MaxCount=1,
    KeyName=ec2_pem_file,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                        {   'Key': 'Name',
                            'Value': 'Docker_Jupyter'
                        }
                    ]
        }
    ],
    InstanceType="t2.micro",
    SecurityGroupIds=[
        Sec_group
    ],
)

Get the instance id of newly created EC2 instance.

In [ ]:
new_instance_id = instances["Instances"][0]["InstanceId"]

Using the instanceId captured above, use describe_instances() method to get instance details which has public DNS address.

In [ ]:
inst_det = ec2.describe_instances(
    InstanceIds=[
        new_instance_id,
    ]
)

Get the public DNS of new instance.

In [ ]:
instance_pub_dns=inst_det["Reservations"][0]["Instances"][0]["PublicDnsName"]
instance_pub_dns

Use the poll function to make instance is completely set up and is ready for use. 

In [ ]:
def poll_until_completed(client, ins_id):
    delay = 2
    while True:
        instance = client.describe_instances(InstanceIds=[ins_id,])
        status = instance["Reservations"][0]["Instances"][0]["State"]["Name"]
#         message = cluster.get('Message', '')
        now = str(datetime.datetime.now().time())
    
        print("instance %s is %s at %s" % (ins_id, status, now))
        if status in ['running','terminated']:
            break

        # exponential backoff with jitter
        delay *= random.uniform(1.1, 2.0)
        time.sleep(delay)

In [ ]:
import random
import time

poll_until_completed(ec2, new_instance_id)  # Can't use it until it's COMPLETED

### SSH through terminal


Just like what we did in [Jupyter_from_Docker](Jupyter_from_Docker.ipynb) lab, we will use the terminal to SSH into the instance and install the the software packages.

Print the keypair file name

In [ ]:
print("keypair file name:",ec2_pem_file)

* Open up a terminal.


* Update the permissions on keypair file

    `Run below cell and copy the output. paste it in the terminal. This will make the keypair file readonly.`

In [ ]:
print("chmod 400 "+os.getcwd()+"/"+ec2_pem_file+".pem")

* Run the cell below and copy the output. 

* Paste the output in terminal and hit enter.

In [ ]:
print("ssh -i "+os.getcwd()+"/"+ec2_pem_file+".pem ec2-user@"+instance_pub_dns)

Paste(right click and paste) the output in terminal and hit enter.

<br>
## <span style="color:red">Note:</span>
<hr size="6" width="100%" noshade style="border-color:#FF0000" align="left">


**Carefully** run all the commands in following sequence as shown in image below. 




```bash


sudo su

yum update –y

yum install -y httpd24 php56 php56-mysqlnd

yum install w3m   (Type y if asking for confirmation)

service httpd start

chown -R ec2-user /var/www/html/

```

What have you done?
 1. Set your PEM (key file) to only be readable by you.
 1. Connected to the remote machine in the Cloud
 1. Become _all powerful, ~~destroyer~~ builder of systems_
 1. Checked for Updates
 1. Installed a package set: httpd24 php56 php56-mysqlnd
   * This includes a web server
 1. Install the w3m package
 1. Start the web server running
 1. Change Ownership of the Web Server's document folder.
---


<h4 style="color:red"> Note: Dont close this terminal window. We will run a command in a while to collect text in the webpage </h4>


Check if the web server is properly installed. Copy and paste the public DNS name of EC2 instance in a browser. 

For example, http://ec2-54-213-80-49.us-west-2.compute.amazonaws.com


-----


![../images/Launch_server/apache_server_test.PNG MISSING](../../module1/images/Launch_server/apache_server_test.PNG)

The Apache test page appears only when there is no content in the document root directory, 
/var/www/html. Since i already added content in that locatin, 
new content will appear at the public DNS address of the EC2 instance instead of the Apache test page.

Below command will configure the web server to start with when ever the system boots,
instead of starting the http service. 
You can do that using the chkconfig command:

    sudo chkconfig httpd on

### Upload the web page Index.php


Upload the html file, "index.php" present in your current working directory into the default web directory `/var/www/html/` of EC2 instance. 

## <span style="color:red">Note: </span>
<hr size="6" width="100%" noshade style="border-color:#FF0000" align="left">


**Replace the public DNS address accordingly with your EC2 instance address in below SCP command.** 


### Upload Index.php

In [ ]:
import os

os.system("scp -o StrictHostKeyChecking=no -i "+
          os.getcwd() +"/" + ec2_pem_file + ".pem "+
          "index.php "+
          "ec2-user@"+instance_pub_dns + ":/var/www/html/")

### Display your instance details


Check if the uploaded **index.php** page is properly displaying on the server. 
Copy and paste the public DNS name of EC2 instance in a browser. 

Example: 
  * http://ec2-54-213-80-49.us-west-2.compute.amazonaws.com/index.php

###  You should see something like this in the browser, 


<img src="../images/static_webpage.PNG">

-----

### Extract the text from webpage

Run below command in the terminal. Make sure to update the public DNS address as shown in the image

Again, replace the public DNS name `ec2-54-213-80-49.us-west-2.compute.amazonaws.com` with your server's name.

    w3m -dump http://ec2-54-213-80-49.us-west-2.compute.amazonaws.com/ > downloaded_file.txt

![../images/webpage_text.PNG MISSING](../../module1/images/webpage_text.PNG)

### Download webpage text

Run below cell to download the text present on the webpage. The text is written to file named downloaded_file.txt on EC2 instance. Below SCP command will download the same file into results folder in current directory. 

In [ ]:
import os

os.system("scp -o StrictHostKeyChecking=no -r -i "+
          os.getcwd() +"/" + ec2_pem_file + ".pem "+
          "ec2-user@"+instance_pub_dns + ":/home/ec2-user/ "+
          os.getcwd()+ "/results/")

**<span style="background:yellow">You should turn off instances after each session</span>** to make sure 
we save free credits and use resources optimally. 
That is the whole point of Cloud computing! 
  * Use resources when they are needed and turn them off when done.

### Delete SSH Keypair

In [ ]:
# Delete SSH Keypair

try:
    os.remove(ec2_pem_file+'.pem')
    print('Local Key Deleted')
except:
    print('Local Key Not Found')
    
response = ec2.delete_key_pair(KeyName=ec2_pem_file)
print('\nAWS Metadata: ')
print('http Status Code : '+str(response['ResponseMetadata']['HTTPStatusCode']))
print('Request ID       : '+response['ResponseMetadata']['RequestId'])
print('Retries          : '+str(response['ResponseMetadata']['RetryAttempts']))

### Terminate the EC2 instance

In [ ]:
ec2.terminate_instances(InstanceIds=[new_instance_id,])

### Delete the security group

Run the polling function to make sure instance is terminated. You cant delete the security group while a running instance is using it. 

In [ ]:
poll_until_completed(ec2, new_instance_id)  # Can't use it until it's COMPLETED

In [ ]:
SG_delete_response = ec2.delete_security_group(
    GroupId=Sec_group,
)

# Save your notebook

<h3><span style="background:red">Be sure to using Git to add the labs/results folder and commit and push this week.</span></h3>

<h1><span style="background:yellow">Deeper/Optional material</span></h1>

To allow ec2-user to manage files in the default root directory for your Apache web server, you need to modify the ownership and permissions of the /var/www directory. We will add a group named www to the EC2 instance which will be given ownership of the /var/www directory and add write permissions. Any members of that group can then add, delete, and modify files for the web server.

#### To set file permissions for the Apache web server

* Add the www group to the EC2 instance with the following command:

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo groupadd www")
print(stdout.read())

* Add the ec2-user user to the www group:

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo usermod -a -G www ec2-user")

* To refresh your permissions and include the new www group, log out:

In [ ]:
stdin, stdout, stderr = client.exec_command("exit")

* Log back in again and verify that the www group exists with the groups command:

In [ ]:
stdin, stdout, stderr = client.exec_command("groups")

* Change the group ownership of the /var/www directory and its contents to the www group:

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo chown -R root:www /var/www")

* Change the directory permissions of /var/www and its subdirectories to add group write permissions and set the group ID on subdirectories created in the future:

In [ ]:
stdin, stdout, stderr = client.exec_command("sudo chmod 2775 /var/www")

stdin, stdout, stderr = client.exec_command("find /var/www -type d -exec sudo chmod 2775 {} +")

* Recursively change the permissions for files in the /var/www directory and its subdirectories to add group write permissions:

In [ ]:
stdin, stdout, stderr = client.exec_command("find /var/www -type f -exec sudo chmod 0664 {} +")

### SSH using Paramiko library

We can SSH into an EC2 instance through AWS CLI in the terminal. 
We can do the same programatically using paramiko python library. 
Paramiko is a Python (2.6+, 3.3+) implementation of the SSHv2 protocol, 
providing both client and server functionality. 

[Click this link if you need reference/digdeeper for paramiko library commands used in below cell](http://docs.paramiko.org/en/2.2/api/client.html#paramiko.client.SSHClient.connect)

Below code cell will SSH into the EC2 instance and install LAMP stack software bundle. 
**Don't run this cell.** We will use a terminal instead.

In [ ]:
# import boto3
# import botocore
# import paramiko
# import time

# client = paramiko.SSHClient()
# client.set_missing_host_key_policy(paramiko.AutoAddPolicy())


# # Connect/ssh to an instance
# try:
#     # hostname is public DNS address of EC2 instance, key_filename is the private key to connect to instance.
#     print("Trying to connect")
#     time.sleep(30)
#     client.connect(hostname=instance_pub_dns, username='ec2-user', key_filename="EC2KeyPair1.pem")
#     print("connected to instance")
    
#     # The -y option in below command installs the updates without asking for confirmation which you normally see for sw installs 
#     print("Update all the existing packages")
#     stdin, stdout, stderr = client.exec_command("sudo yum update –y")
#     data = stdout.read().splitlines()
#     for line in data:
#         print('stdout: ',line)

# #     print('sudo yum update –y  stdout: ',stdout)
#     print('sudo yum update –y  stderr: ', stderr)
    
    
#     # Install the Apache web server with the PHP software package. Below command installs multiple software packages 
#     # and related dependencies at the same time:
#     print("Installing the apache web server, PHp and MySQL")
#     stdin, stdout, stderr = client.exec_command("sudo yum install -y httpd24 php56 php56-mysqlnd")
#     for line in data:
#         print('stdout: ',line)
    
#     print('sudo yum install -y httpd24 php56 php56-mysqlnd  stderr: ', stderr)
    
    
#     # Start the web server with the below command
#     print("Starting httpd service")
#     stdin, stdout, stderr = client.exec_command("sudo service httpd start")
#     for line in data:
#         print('stdout: ',line)
#     print('sudo service httpd start  stderr: ', stderr)
    
    
    
# except Exception as e:
#     print(e)

### Uploading Index.php

Move to the default web directory and upload the html file in the local machine to EC2 instance using scp as shown below

In [ ]:
# import boto3
# import botocore
# import paramiko
# import time

# client = paramiko.SSHClient()
# client.set_missing_host_key_policy(paramiko.AutoAddPolicy())


# # Connect/ssh to an instance
# try:
#     # hostname is public DNS address of EC2 instance, key_filename is the private key to connect to instance.
#     print("Trying to connect")
#     time.sleep(30)
#     client.connect(hostname=instance_pub_dns, username='ec2-user', key_filename="EC2KeyPair1.pem")
#     print("connected to instance")
    
#     stdin, stdout, stderr = client.exec_command("cd /var/www/html")
    
#     stdin, stdout, stderr = client.exec_command("sudo chown -R ec2-user /var/www/html/")
    
#     sftp = client.open_sftp()
#     sftp.put('index.php', '/var/www/html/index.php')
    
    
# except Exception as e:
#     print(e)

# Save your notebook, then `File > Close and Halt`